# 最低限のプロミスを自作して理解する

- プロミスのレゾルバはプロミスを返さない
- Promiseコンストラクタのレゾルバやthenのコールバックはリジェクトに対応しない

という仮定のもとで最低限のなんちゃってプロミスを自作して理解する．

## 最低限のプロミスの定義

In [ ]:
function Promise0(resolver) {
    this.fulfilled = false;
    this.value = undefined;
    this.waiting = [];
    // resolveはfoo.resolveのようには呼ばれないので
    // thisをあらかじめbindしておく
    resolver(this.resolve.bind(this));
}

Promise0.prototype.resolve = function (result) {
    if (this.fulfilled) return;
    this.fulfilled = true;
    // resolveにはプロミスは渡されないと仮定
    this.value = result;
    for (callback of this.waiting) {
        callback(this.value);
    }
    this.waiting = [];
};
    
Promise0.prototype.done = function (callback) {
    if (this.fulfilled) {
        let that = this;
        // コールバック関数callbackが非同期に実行されることを保証
        setTimeout(function() { callback(that.value); }, 0); return; 
    }
    this.waiting.push(callback);
};
    
Promise0.prototype.then = function (onFulfilled) {
    let that = this;
    return new Promise0(function (resolve) {
        that.done(function (result) {
            let p = onFulfilled(result);
            // thenのコールバックonFulfilledがプロミスを返すことを保証
            if (! (p instanceof Promise0)) { p = pure(p); }
            p.done(resolve);
        });
    });
};

// 値をラップしただけのプロミス
Promise0.resolve = function (value) {
    return new Promise0(function (resolve) { resolve(value); });
}

## テスト例

以下の例はプロミスが生成時に直ちに解決されるときでも非同期に実行されることの確認

In [ ]:
console.log("BEGIN");
Promise0.resolve(1234).done(console.log);
console.log("END");

逐次実行，さらにコールバックはプロミス値を返すことの確認

In [ ]:
console.log("BEGIN");
Promise0.resolve(1)
    .then(function (n) { console.log(n); return 2; })
    .then(function (n) { console.log(n); return 3; })
    .then(function (n) { console.log(n); });
console.log("END");

ms秒後にサイコロをふる

In [ ]:
function roll(ms) {
    return new Promise0(function(resolve) {
        setTimeout(function () {
            resolve(Math.floor(Math.random() * 6 + 1));
        }, ms);
    });
}

In [ ]:
roll(1000).done(console.log);

In [ ]:
roll(1000).then(function (n) { console.log(n); return roll(1000); })
          .then(function (n) { console.log(n); return roll(1000); })
          .then(function (n) { console.log(n); })    // .done(console.log)でも可

## `async`/`await`構文で使うテスト

`async`/`await`はthenableなオブジェクト一般に対応している．

In [ ]:
a_thenable = { then(onFulfilled) { return onFulfilled(123); } };

(async function () {
    console.log(await a_thenable);
})();

In [ ]:
a_thenable = { then(onFulfilled, onRejected) { return onRejected(123); } };

(async function () {
    try {
        await a_thenable;
    } catch (n) {
        console.log(n);
    }
})();

なので自作のなんちゃってプロミスにも使えるはず．

In [ ]:
console.log("BEGIN");
(async function() {
    console.log(await roll(1000));
    console.log(await roll(1000));
    console.log(await roll(1000));    
})();
console.log("END");

In [ ]:
console.log("BEGIN");
(async function() {
    for (;;) {
        n = await roll(1000);
        console.log(n);
        if (n == 1) { console.log("STOP"); break; }
    }
})();
console.log("END");